<a href="https://colab.research.google.com/github/nghianh993/BDS/blob/master/B%E1%BA%A3n_sao_c%E1%BB%A7a_Colaboratory_ch%C3%A0o_m%E1%BB%ABng_b%E1%BA%A1n!.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DE

In [ ]:
import numpy as np
from datetime import datetime, timedelta
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

# Dữ liệu mẫu
data = [["2023-12-13",["91706","82736","43914","21410","62100","28841","54653","02003","00059","46346","3044","7942","3115","2346","2092","6475","3946","3685","3452","1207","346","956","814","55","83","76","44"]],
        ["2023-12-12",["62970","39467","40628","04048","85828","50051","17798","87941","64115","64474","6711","8324","0621","5607","1058","6146","6328","5593","5917","7918","954","545","397","63","29","28","51"]],
        ["2023-12-11",["38038","02475","03846","53934","90956","89781","13882","01561","73501","93779","8318","6981","8615","9513","8328","3837","0107","9532","9879","1742","729","766","938","85","12","33","48"]]]

# Chuyển đổi dữ liệu thành mảng có hình dạng (số lượng mẫu, timesteps, features)
timesteps = 3  # Số lượng ngày liên tiếp
features = 27  # Số lượng số lựa chọn trong xổ số
num_samples = len(data)

X = np.zeros((num_samples, timesteps, features))
y = np.zeros((num_samples, 1))
for i, sample in enumerate(data):
    numbers = sample[1]
    date_str = sample[0]
    date = datetime.strptime(date_str, "%Y-%m-%d")
    for j in range(timesteps):
        prev_date = date - timedelta(days=j)
        prev_date_str = prev_date.strftime("%Y-%m-%d")
        for k in range(features):
            if prev_date_str == date_str and j < len(numbers):
                X[i, j, k] = int(numbers[j][k])
    # Lưu trữ kết quả xổ số (số cuối cùng trong danh sách số)
    y[i] = int(numbers[-1])

# Chuẩn hóa dữ liệu
X = X / 100000.0  # Chuẩn hóa các số lựa chọn trong khoảng từ 0 đến 1
y = y / 100000.0

# Tạo mô hình
model = Sequential()
model.add(LSTM(128, input_shape=(timesteps, features), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(64, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(32, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(16))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))

# Biên dịch mô hình
model.compile(loss='mean_squared_error', optimizer='adam')

# Huấn luyện mô hình
model.fit(X, y, epochs=100, batch_size=32, verbose=1)

# Dự đoán kết quả
X_prediction = X[-1:]  # Dữ liệu dự đoán là mẫu cuối cùng trong tập huấn luyện
predicted_numbers = model.predict(X_prediction)
predicted_numbers = predicted_numbers * 100000

print("Kết quả dự đoán xổ số:", predicted_numbers[0][0])

IndexError: string index out of range

# LO

In [ ]:
import requests
import numpy as np
from datetime import datetime, timedelta
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from google.colab import drive
import pickle
drive.mount('/content/drive')
model_path = '/content/drive/MyDrive/AIKQXS.pkl'
url = "https://rongbachkim.com"
now = datetime.now()
data = []
minusDay = 300
next_date = now + timedelta(days=minusDay)
for number in range(1, 3):
    next_date = next_date - timedelta(days=minusDay)
    # Gửi yêu cầu HTTP đến API và nhận dữ liệu JSON
    response = requests.get(url + '/ketqua.php?getkq&ngay=' + next_date.strftime("%Y-%m-%d") + '&days='+ str(minusDay) +'&wday=0')
    data = data + response.json()
# Chuyển đổi dữ liệu thành đặc trưng (features) và kết quả (labels)
print(data)
features = []
labels = []
for day in data:
    date = day[0].replace('-', '')
    results = day[1]
    for result in results:
        if len(result) > 0:
            features.append([float(result)])
            labels.append([float(date)])

features = np.array(features[::-1], dtype=np.float32)
labels = np.array(labels[::-1], dtype=np.float32)
results = []
for number in range(1, 2):
    print('Đang chạy tới: ', number)
    with open(model_path, 'rb') as file:
        model = pickle.load(file)
    # Huấn luyện mô hình
    model.fit(features, labels, epochs=10, batch_size=27)
    #Lưu lại model đã huấn luyện
    with open(model_path, 'wb') as file:
        pickle.dump(model, file)
    X_next_day = np.array([features[-1]])
    y_predicted = model.predict(X_next_day)
    # Chuyển đổi kết quả dự đoán thành kiểu nguyên
    y_predicted = y_predicted.flatten().astype(int)
    # Tạo kết quả dưới dạng mảng tương tự như dữ liệu đã cho
    next_results = [str(num).zfill(5) for num in y_predicted]
    for rs in next_results:
        results.append(rs)
        print(rs)
# Thống kê kết quả
new_results = []
for result in results:
    new_results.append(result[-2::])

print(new_results)

counts = {}
for lottery in new_results:
    if lottery in counts:
        counts[lottery] += 1
    else:
        counts[lottery] = 1
sortCounts = sorted(counts.items(), key=lambda x: x[1], reverse=True)
# In kết quả dự đoán đầy đủ
print([now.strftime("%Y-%m-%d"), results])
# In kết quả dự đoán lô tô
print([now.strftime("%Y-%m-%d"), new_results])
# In kết quả dự đoán thống kê số lượng
print([now.strftime("%Y-%m-%d"), sortCounts])